In [1]:
from libraries import *
from parameters import *
import statsmodels.api as sm
import sklearn.feature_selection
import statsmodels.api as sm
import sklearn.feature_selection
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings('ignore')
from statannot import add_stat_annotation
from numpy.random import seed
from numpy.random import randint
import statsmodels.formula.api as smf

import statsmodels.api as sm
%load_ext rpy2.ipython

import sys
# setIndex_1=sys.argv[1]
# setIndex_2=sys.argv[2]


/home/beraslan/miniconda3/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
setIndex_1=0

In [3]:
os.getcwd()
os.chdir(projectDir)

In [4]:
setIndex_2= setIndex_1+10
print(setIndex_2)

10


In [5]:
adataC = sc.read('outputs/anndata/adata-hash-features_SingleKO_ForSCVI_09072021.h5ad')

allGeneNames = pd.read_csv("/home/beraslan/jovian-work/analysisSingle/AllExpGeneNames.csv", index_col=0)

# adataC.obs["subCellType"] = "DC2"
# adataC.obs.loc[adataC.obs.leiden.isin(['3']), "subCellType"] = "MacDC"
# adataC.obs.loc[adataC.obs.leiden.isin(['8']), "subCellType"] = "DC1"
# adataC.obs.loc[adataC.obs.leiden.isin(['5']), "subCellType"] = "MReg"

In [6]:
guideModules = pd.read_csv("/home/beraslan/jovian-work/analysisSingle/ME_GuideModules_leiden_6_Modules.csv", index_col=0)
guideModules.index = "GENE_"+guideModules.GuideName+"_"
guideModules.GuideName = "GENE_"+guideModules.GuideName+"_"

In [7]:
guideModules

,GuideName,GuideGroup,GuideColor
GuideName,,,
GENE_Aamp_,GENE_Aamp_,4,#aa40fc
GENE_Bop1_,GENE_Bop1_,4,#aa40fc
GENE_Cirh1a_,GENE_Cirh1a_,4,#aa40fc
GENE_Dcaf13_,GENE_Dcaf13_,4,#aa40fc
GENE_Grb2_,GENE_Grb2_,4,#aa40fc
...,...,...,...
GENE_Wdr20_,GENE_Wdr20_,2,#279e68
GENE_Wdr26_,GENE_Wdr26_,2,#279e68
GENE_Wdr33_,GENE_Wdr33_,2,#279e68


In [ ]:
covariates = guideModules.GuideName
covariates = covariates[covariates != "GENE_CONTROL_"]

In [ ]:
guideMatrix = adataC.obs[covariates]
guideMatrix.shape
guideMatrix.rename(columns = {'GENE_Rnf8-cmtr1_':'GENE_Rnf8cmtr1_'}, inplace = True) 
guideMatrix.rename(columns = {'GENE_Siah1-ps1_':'GENE_Siah1ps1_'}, inplace = True) 
guideMatrix.rename(columns = {'GENE_Siah1-ps2_':'GENE_Siah1ps2_'}, inplace = True) 

KOGenes = pd.Series(guideMatrix.columns)

In [ ]:
## Add covariates that should be corrected for
df = adataC.obs[["n_genes", "mt_frac", "leiden" ]]

guideMatrix = guideMatrix.join(df)
guideMatrix.shape

In [ ]:
expressionMatrix = pd.DataFrame(adataC.raw.X.A)
expressionMatrix.columns = allGeneNames.iloc[:,0]
expressionMatrix = expressionMatrix[adataC.var_names]
expressionMatrix.index = adataC.obs.index

In [ ]:
selColumns = pd.Series(KOGenes[setIndex_1:setIndex_2]).append(pd.Series(["n_genes", "mt_frac", "leiden"]))

In [ ]:
controlExpressionMat = expressionMatrix.loc[adataC.obs["GENE_CONTROL_"] == 1,]
controlGuideMat = guideMatrix.loc[adataC.obs["GENE_CONTROL_"] == 1,]
controlExpressionMat = controlExpressionMat.iloc[0:5000,:]
controlGuideMat = controlGuideMat.iloc[0:5000,:]
controlGuideMat=controlGuideMat[selColumns]

In [ ]:
controlGuideMat

In [ ]:
geneExpressionMat = expressionMatrix.loc[adataC.obs[covariates[setIndex_1:setIndex_2]].sum(axis=1) > 0,]
geneGuideMat = guideMatrix.loc[adataC.obs[covariates[setIndex_1:setIndex_2]].sum(axis=1) > 0,]
geneGuideMat=geneGuideMat[selColumns]

In [ ]:
allExpMat = np.concatenate((controlExpressionMat, geneExpressionMat))
allGuideMat = controlGuideMat.append(geneGuideMat)

In [ ]:
allGuideMat.shape

In [ ]:
controlGuideMat.shape

In [ ]:
geneGuideMat.shape

In [ ]:
allVars = pd.Series(KOGenes[setIndex_1:setIndex_2]) 
allVarsInteract = KOGenes[setIndex_1:setIndex_2] + "*leiden"

myForm = allVars.append(allVarsInteract)
myForm = myForm.append(pd.Series(["n_genes","mt_frac","leiden"]))

myFormula = "+".join(myForm)
my_formula = "y~" + myFormula

In [ ]:
allResp=adataC.var_names
allResp

In [ ]:
del adataC
del expressionMatrix

In [ ]:
%%R -i allGuideMat,allExpMat,my_formula,allResp,setIndex_1,setIndex_2  -w 8 -h 4 -u in
library("MASS")
library("broom")

for(i in seq(1,ncol(allExpMat),20)){
    print(i)
    
    coefDF = data.frame()
    for(j in seq(i,i+20,1)){
       
        
      tryCatch(
            expr = {
                   allGuideMat["y"] = allExpMat[,j]
                   myFit <- glm.nb(formula(my_formula), data = allGuideMat)
                   myDF <- data.frame(tidy(myFit))
                   myDF$respGene = allResp[j] 
                   coefDF <- rbind(coefDF, myDF)
            },
            error = function(e){ 
                print(e)
            },
            finally = {
                # (Optional)
                # Do this at the end before quitting the tryCatch structure...
            }
      )
     
    }
    
     saveRDS(coefDF, paste0("./ClusterInteractions/coefs_",setIndex_1,"_",setIndex_2,"_", i, "_",i+20,".rds"))
    

}

In [ ]:
%%R  -w 8 -h 4 -u in

readRDS("./ClusterInteractions/")